**Sets directories:**
   - **anapyze_dir:** Directory where the anapyze_directory is located
   - **dir_patients:** The directory resulting from 0_Reorder_Data or 0_Reorder_ADNI

In [19]:
import warnings

warnings.filterwarnings("ignore")

import os, sys
import nibabel as nib
from nibabel import processing as nibproc
from os.path import join, exists, isdir
import numpy as np

anapyze_dir = r'/Users/jsilva/repositories/anapyze'
anapyze_rsc = join(anapyze_dir, 'resources')
sys.path.insert(0, anapyze_dir)

from spm import SPM

dir_patients = r'/Volumes/txusser_data/IBIS_DATA/Reorder_New/'

**Sets directories:**
  
   - **template_pet:** Different templates are available at anapyze/resource/templates/ or you can bring your own
   - **run_spm_file_in_Python:** If you have configured the paths from spm in spm_proc, you can set this to true. Othwerwise you must run the created normalize.m in MATLAB with SPM

In [20]:
# Put your data directory here
# Change your templates here if necessary
template_pet = join(anapyze_rsc, 'templates', 'FDG_PET_template.nii')

spm_proc = SPM(r'/Users/jsilva', r'/Users/jsilva')
run_spm_file_in_Python = False

if template_pet[-2:] == 'gz':
    img_ = nib.load(template_pet)
    data_ = img_.get_fdata()
    out = nib.Nifti1Image(data_, img_.affine, img_.header)
    nib.save(out, template_pet[0:-3])
    print(f"Decompressed: {template_pet}. SPM does not accept .nii.gz files...")
    template_pet = template_pet[0:-3]

list_dirs = os.listdir(dir_patients)

#This will create a cat_12.m file that must be run on MATLAB separately

images = []

for i in list_dirs:
    
    dir_subj = join(dir_patients, i)
    
    if isdir(dir_subj):

        print('\nProcessing %s: Subject %s/%s' % (i, list_dirs.index(i), len(list_dirs)))
    
        list_subj = os.listdir(dir_subj)
    
        fdg_list = []
    
        for j in list_subj:
            if j.startswith('fdg_'):
                fdg_list.append(j)
    
        for k in fdg_list:
            
            pet_dir = join(dir_patients, i, k)
            
            check_cat_processing = [join(pet_dir, 'wpet.img')]
        
            if all(exists(j) for j in check_cat_processing):
                pass
        
            else:
        
                pet_in = join(pet_dir, 'pet.img')
        
                if exists(pet_in):
                    images.append(pet_in)
    
dir_proc = os.getcwd()
    
norm_spm = spm_proc.normalize_multiple_pets(dir_proc, images, template_pet, cutoff=15, nits=16, reg=1,
                                                preserve=0, affine_regularization_type='mni', source_image_smoothing=8,
                                                template_image_smoothing=3, bb=None,
                                                write_vox_size='[1.5 1.5 1.5]', wrapping=True, interpolation=4, prefix='w',
                                                run=run_spm_file_in_Python)

In [21]:
for i in list_dirs:

    dir_subj = join(dir_patients, i)

    if isdir(dir_subj):

        print('\nProcessing %s: Subject %s/%s' % (i, list_dirs.index(i), len(list_dirs)))

        list_subj = os.listdir(dir_subj)

        fdg_list = []

        for j in list_subj:
            if j.startswith('fdg_'):
                fdg_list.append(j)

        for k in fdg_list:
            
            pet_in = join(dir_subj, k, 'wpet.hdr')
            
            if exists(pet_in):
                siemens_file = join(dir_subj, k, '1080')
                if exists(siemens_file):
                    smoothing = 2.5
                else:
                    smoothing = 6.5

                img_ = nib.load(pet_in)
                smoothed_pons = nibproc.smooth_image(img_, smoothing, out_class=nib.Nifti1Image)
                data_ = smoothed_pons.get_fdata()

                smoothed_output = join(dir_subj, k, 'swpet.hdr')
                out_img = nib.Nifti1Image(data_, img_.affine, img_.header)
                nib.save(out_img, smoothed_output)
            else:
                print('wpet MISSING!!!!')
            
            

The following cell will apply intensity normalization using the Pons as a Reference region. Two images will be created, wpet_normpons.hdr and swpet_normpons.hdr (smoothed to smoothing_fwhm)

In [22]:
# Normalization using a pons reference region
list_dirs = os.listdir(dir_patients)

reference_region = join(anapyze_rsc, 'reference_regions', 'Pons_MNI_Ewers_1.5mm.nii.gz')
ref_data = nib.load(reference_region).get_fdata()
pons_vox = np.where(ref_data == np.unique(ref_data)[1])

for i in list_dirs:

    dir_subj = join(dir_patients, i)
    
    if isdir(dir_subj):
    
        print('\nProcessing %s: Subject %s/%s' % (i, list_dirs.index(i), len(list_dirs)))
    
        list_subj = os.listdir(dir_subj)
    
        fdg_list = []
    
        for j in list_subj:
            if j.startswith('fdg_'):
                fdg_list.append(j)
    
        for k in fdg_list:
            pet_in = join(dir_subj, k, 'swpet.hdr')

            if exists(pet_in):
                img_ = nib.load(pet_in)
                data_ = img_.get_fdata()
        
                ref_val = np.nanmean(data_[pons_vox])
                norm_data = data_ / ref_val
        
                norm_data = np.nan_to_num(norm_data, nan=0)
                norm_data[norm_data < 0] = 0
        
                pet_out = join(dir_subj, k, 'swpet_pons_norm.hdr')
                out_img = nib.Nifti1Image(norm_data, img_.affine, img_.header)
                nib.save(out_img, pet_out)

The following cell will apply intensity normalization using the histogram intensity normalization as published by Lopez-Gonzalez et al. (Neuroimage 2020).
This will create an image swpet_normhistogram.hdr
This method is theoretically superior to the reference region one.


In [23]:
# Normalization using a data-driven method
list_dirs = os.listdir(dir_patients)
smoothing_fwhm = 4

mask_path = join(anapyze_rsc, 'mask', 'gm_cat12.nii.gz')
mask_img_ = nib.load(mask_path)

if len(mask_img_.shape) == 4:
    mask_data = mask_img_.get_fdata()[:, :, :, 0]
else:
    mask_data = mask_img_.get_fdata()[:, :, :]

mask_indx = np.where(mask_data == 1)

template_img_ = nib.load(template_pet)
smoothed_template = nibproc.smooth_image(template_img_, smoothing_fwhm, out_class=nib.AnalyzeImage)
template_data = smoothed_template.get_fdata()
mean_template = np.mean(template_data[mask_indx])

for i in list_dirs:

    dir_subj = join(dir_patients, i)

    if isdir(dir_subj):

        print('\nProcessing %s: Subject %s/%s' % (i, list_dirs.index(i), len(list_dirs)))

        list_subj = os.listdir(dir_subj)

        fdg_list = []

        for j in list_subj:
            if j.startswith('fdg_'):
                fdg_list.append(j)

        for k in fdg_list:
            pet_in = join(dir_subj, k, 'swpet.hdr')

            if exists(pet_in):
                img_ = nib.load(pet_in)
                data_ = img_.get_fdata()
        
                mean_fdg = np.mean(data_[mask_indx])
                fdg_data = data_ * (mean_template / mean_fdg)
        
                division = template_data[mask_indx] / fdg_data[mask_indx]
                values, bins = np.histogram(division, 200, range=[0.5, 2])
                amax = np.amax(values)
                indx = np.where(values == amax)
                norm_value = bins[indx][0]
                fdg_data = fdg_data * norm_value
        
                fdg_data = np.nan_to_num(fdg_data, nan=0)
                fdg_data[fdg_data < 0] = 0
        
                pet_out = join(dir_subj, k, 'swpet_hist_norm.hdr')
        
                img = nib.Nifti1Image(fdg_data, img_.affine, img_.header)
                nib.save(img, pet_out)


In [24]:
pet_dir